# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [2]:
import os
import pandas as pd
import numpy as np
import re
from __future__ import print_function
PUIdata = os.getenv('PUIDATA')

In [ ]:
# Load dataset
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

In [ ]:
data.columns

In [ ]:
data.head()

# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [ ]:
# Check if all Boroughs and Community Districts are represented in the Data 
UniqBoroDistricts = data['Community Board'].unique()
print (UniqBoroDistricts)

# How many unique values do we have? 
print ("Number of Community Boards: ", len(UniqBoroDistricts))
#There are 6 unspecified community boards, one for each boro and one general.

In [ ]:
UniqBoroDistricts.sort()
print (UniqBoroDistricts)

In [ ]:
# Q: Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.

# A: There are community boards included in the data that don't actually exist. 
# All districts above 18 are invalid, along with the "unspecified" community boards. 

UniqComplaints = len(data['Unique Key'].unique())
NumUniqueKey = len(data['Unique Key'])
difference = NumUniqueKey - UniqComplaints

In [ ]:
# Check for duplicates? Are these plausible?
print ("Number of \"Unique Key\" in original data set: ", len(data['Unique Key']))
print ("Number of \"Unique Key\" entries in data set after de-duplication: ", UniqComplaints)
print ("Difference: ", difference)
print ("Number of Duplicates: ", difference*2)

#New 
#data['doubles'] = [[data['Unique Key'].value_counts() > 1]]

In [ ]:
#data=data.set_index(['Unique Key'])
#data.head()
#data['doubles'] = data.index.value_counts() > 1
#print (difference)
data=data.drop_duplicates(subset='Unique Key', keep='last')
#data[data.doubles == True]
#print (difference)
data.head()

In [ ]:
#Confirming that duplicate drop worked correctly.
print ("Data row count following duplicates drop: ", len(data)) #Success.

In [ ]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
# -----------------------------------------------------
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
# -----------------------------------------------------

#Using Scott's code to create a marker for erroneous community board
data["valid_board"] = np.ones(len(data.Borough), np.float)
data['Complaints'] = np.zeros(len(data.valid_board))
com_grp = data[['Community Board', 'Borough', 'Complaints']].groupby(["Community Board",'Borough'])
com_dists = com_grp.agg('count')


for bname in com_dists.itertuples():
    if re.match(r"[01]\d", bname[0][0]) == None:
        data.valid_board.loc[data['Community Board'] == bname[0][0]] = np.nan
data.valid_board.isnull().sum() #output sum of all NaN values. 

In [ ]:
com_dists #Checking community districts.

In [ ]:
# Drop the marked districts
data = data[~data['valid_board'].isnull()]
len(data)

In [ ]:
# Produce your result: Generate an indicator which ranks the Community District by
# complaint numbers on the Community district level
data['Complaints'] = np.zeros(len(data.valid_board))
com_grp = data[['Community Board', 'Borough',
                    'Complaints']].groupby(['Community Board', 'Borough'])
comp_by_board = com_grp.agg({'Complaints' : 'count'})
comp_by_board.head()

In [ ]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
comp_by_board.sort('Complaints', ascending=False).head()

In [ ]:
# Safe reduced data frame (Community District level) above.

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [7]:
# Population by Community District
df_Demog=pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')
df_Demog.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [8]:
# Check variables in file
#Printing all column titles to identify variables in data set:
for x in df_Demog.columns:
    print(x)

FIPS
cd_id
Total Population
Population Density (per sq. mile)
% Total Population: Male
% Total Population: 18 to 24 Years
% Total Population: 25 to 34 Years
% Total Population: 35 to 44 Years
% Population 5 Years And Over: Speak Only English
% Population 5 Years And Over: Spanish or Spanish Creole
% Population 5 Years And Over: Spanish or Spanish Creole: Speak English "very Well"
% Population 5 Years And Over: Spanish or Spanish Creole: Speak English Less Than "very Well"
Population 25 Years and over:
Population 25 Years and over: Less Than High School
Population 25 Years and over: High School Graduate (includes equivalency)
Population 25 Years and over: Some college
Population 25 Years and over: Bachelor's degree
Population 25 Years and over: Master's degree
Population 25 Years and over: Professional school degree
Population 25 Years and over: Doctorate degree
% Population 25 Years and over: Less Than High School
% Population 25 Years and over: High School Graduate (includes equivalen

In [10]:
# How many community districts are in file? 
print("Number of community districts in file: ", len(df_Demog.cd_id.unique()))

Number of community districts in file:  59


In [45]:
#Renaming column names for ease of referencing. 
TotalPop = "Total Population"
PopDens = "Population Density (per sq. mile)"
PctBach = "% Population 25 Years and over: Bachelor's degree or more"
PctMaster = "% Population 25 Years and over: Master's degree or more"
PctProf = "% Population 25 Years and over: Professional school degree or more"
PctDoc = "% Population 25 Years and over: Doctorate degree.1"

In [76]:
#Income:
#Middle class in NYC, according to NYTIMES: $45,000 and $134,000
#http://www.nytimes.com/2013/01/20/realestate/what-is-middle-class-in-manhattan.html
#Extrapolated Lower & Upper accordingly. 

#Language:
df_Demog['% English Speakers'] =  (df_Demog['% Population 5 Years And Over: Speak Only English'] +
                                df_Demog['% Population 5 Years And Over: Spanish or Spanish Creole: Speak English "very Well"'])

#Lower Income: 
df_Demog['% Low Income'] = (df_Demog["% Households: Less than $10,000"] +
                        df_Demog["% Households: $10,000 to $14,999"] +
                        df_Demog["% Households: $15,000 to $19,999"] +
                        df_Demog["% Households: $20,000 to $24,999"] +
                        df_Demog["% Households: $25,000 to $29,999"] +
                        df_Demog["% Households: $30,000 to $34,999"] +
                        df_Demog["% Households: $35,000 to $39,999"] +
                        df_Demog["% Households: $40,000 to $44,999"])


df_Demog['% Middle Income'] = (df_Demog["% Households: $45,000 to $49,999"] + 
                             df_Demog["% Households: $50,000 to $59,999"] + 
                             df_Demog["% Households: $60,000 to $74,999"] + 
                             df_Demog["% Households: $75,000 to $99,999"] + 
                             df_Demog["% Households: $100,000 to $124,999"] + 
                             df_Demog["% Households: $125,000 to $149,999"])

df_Demog['% Upper Class'] = (df_Demog["% Households: $150,000 to $199,999"] + 
                           df_Demog["% Households: $200,000 or More"])

#Education
df_Demog['% High School Graduate']=df_Demog['% Population 25 Years and over: High School Graduate or more (includes equivalency)']
df_Demog['% College Educated'] = df_Demog['% Population 25 Years and over: Bachelor\'s degree or more']
df_Demog['% Advanced Degree'] = (df_Demog["% Population 25 Years and over: Master's degree or more"] +  
                                df_Demog["% Population 25 Years and over: Doctorate degree.1"])
df_Demog['% Professional Degree'] = df_Demog["% Population 25 Years and over: Professional school degree or more"]

#Employment:
df_Demog['% Employment Rate'] = (df_Demog['% Employed Civilian Population 16 Years And Over: Private Sector'] + 
                               df_Demog['% Employed Civilian Population 16 Years And Over: Public Sector'] + 
                               df_Demog['% Employed Civilian Population 16 Years And Over: Self-Employed (incorporated and not incorporated)'])    

df_Demog_slim = df_Demog[["FIPS", "cd_id", TotalPop, PopDens, "% High School Graduate", "% College Educated", "% Advanced Degree", "% Professional Degree",
                          "% English Speakers", "% Employment Rate", "Low Income", "Middle Income", "Upper Class"]]

In [78]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
df_Demog_slim
# Save data frame

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% High School Graduate,% College Educated,% Advanced Degree,% Professional Degree,% English Speakers,% Employment Rate,Low Income,Middle Income,Upper Class
0,3603701,BX08,106737,31229.950060,83.07,39.87,21.28,7.24,70.81,87.46,44.12,46.58,9.30
1,3603702,BX12,134644,19966.678390,80.22,23.44,6.37,0.24,85.21,88.58,50.05,43.91,6.04
2,3603703,BX10,121209,12913.817030,82.66,27.17,11.27,1.58,81.30,89.49,41.67,50.40,7.92
3,3603704,BX11,135839,35677.954530,77.16,24.11,12.24,3.39,66.00,90.63,47.73,44.02,8.24
4,3603705,BX03,172247,39405.792220,61.98,11.87,4.59,1.26,66.91,92.49,76.07,22.85,1.07
5,3603705,BX06,172247,39405.792220,61.98,11.87,4.59,1.26,60.07,92.49,76.07,22.85,1.07
6,3603706,BX07,135893,86487.077920,67.09,17.98,4.89,1.06,59.44,88.45,61.14,37.40,1.47
7,3603707,BX05,132850,87974.348600,64.03,12.64,4.79,0.84,58.63,91.85,77.97,21.12,0.89
8,3603708,BX04,141467,71270.882190,62.98,16.26,4.66,0.60,69.86,90.72,70.16,28.17,1.67
9,3603709,BX09,190126,42752.506900,69.66,16.09,4.82,0.46,62.69,92.67,58.70,36.95,4.35


In [79]:
# Infrastructure by Community District
df_infr = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [80]:
# Check variables in file
for x in df_infr.columns: 
    print(x)

FIPS
Geographic Identifier
Qualifying Name
Households
Households: With An Internet Subscription
Households: Dial-Up Alone
Households: Dsl
Households: With Mobile Broadband
Households: Without Mobile Broadband
Households: Cable Modem
Households: With Mobile Broadband.1
Households: Without Mobile Broadband.1
Households: Fiber-Optic
Households: With Mobile Broadband.2
Households: Without Mobile Broadband.2
Households: Satellite Internet Service
Households: With Mobile Broadband.3
Households: Without Mobile Broadband.3
Households: Two or More Fixed Broadband Types, or Other
Households: With Mobile Broadband.4
Households: Without Mobile Broadband.4
Households: Mobile Broadband Alone or With Dialup
Households: Internet Access Without A Subscription
Households: No Internet Access
% Households: With An Internet Subscription
Households.1
Households: Has A Computer
Households: With Dial-Up Internet Subscription Alone
Households: With A Broadband Internet Subscription
Households: Without An Inter

In [81]:
# How many community districts are in file? 
print ("Number of community districts in file: ", len(df_infr))

Number of community districts in file:  59


In [86]:
#Exploring Community districts.
#for x in (df_infr['Qualifying Name']): 
#    print(x)
    
#for x in (df_infr['Geographic Identifier']):
#    print(x)

In [89]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data

df_infr["cd_id"] = df_infr["Geographic Identifier"]
df_infr[["Qualifying Name", "Households", "Households: With An Internet Subscription", "cd_id"]]

,Qualifying Name,Households,Households: With An Internet Subscription,cd_id
0,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,79500US3603701
1,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,79500US3603702
2,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,79500US3603703
3,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,79500US3603704
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,79500US3603705
5,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,79500US3603705
6,"NYC-Bronx Community District 7--Bedford Park, ...",47252,31468,79500US3603706
7,NYC-Bronx Community District 5--Morris Heights...,44699,26332,79500US3603707
8,"NYC-Bronx Community District 4--Concourse, New...",47935,29376,79500US3603708
9,"NYC-Bronx Community District 9--Castle Hill, N...",64011,45976,79500US3603709


In [ ]:
# Aggregate internet type by high and low connections

In [ ]:
# Save data frame 

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [ ]:
# Harmonize identifier of dataframe 1

In [ ]:
# Harmonize identifier of dataframe 2

In [ ]:
Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('BX', 'BRONX '))
Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('MN11111', 'MN11'))
Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('MN', 'MANHATTAN '))
Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('QN', 'QUEENS '))
Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('SI', 'STATEN_ISLAND '))
Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('BK', 'BROOKLYN '))
Demog.head()

In [ ]:
#Demog['splits']=Demog['cd_id'].str.split(' ')
#find=Demog.index.get_loc?
#flip = lambda x: Demog['splits'][find][1] + " " + Demog['splits'][find][0]
#Demog['boro']=Demog['splits'].apply(flip)
Demog['boro']=np.zeros(len(Demog)) 

for i in range (0,len(Demog)):
    Demog['boro'][i] = (Demog['cd_id'].str.split(' '))[i][1] + ' ' + (Demog['cd_id'].str.split(' '))[i][0]

Demog['cd_id']=Demog['cd_id'].map(lambda x: x.replace('STATEN_ISLAND', 'STATEN ISLAND '))

Demog.head()

In [ ]:
data['Community Board']=data['Community Board'].rename('cd_id')

data.rename(columns={'Community Board':'cd_id'}, inplace=True)
#data.columns #Column name changed successfully.

In [ ]:
# Harmonize identifier of dataframe 3

In [ ]:
# Link the 3 dataframes

In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?